# Simple Binary Connect
A bare-bones instructional implementation of [BinaryConnect(2015) by Matthieu Courbariaux, Yoshua Bengio, Jean-Pierre David](https://arxiv.org/pdf/1511.00363). The rest of the post is organised as follows:

1. Introduction (contents of the README)
2. Objective
3. Some Theory
4. Forward Pass
5. Backward Pass
6. Testing
7. Adding Binarization
8. Testing
9. Discussion

## 1. Introduction
A conversation at work spawned a discussion about the difficulties of successfully deploying Deep Neural Networks on embedded and mobile devices. A [very]rough review reveals three broad approaches:
- Training shallower/smaller architectures to reduce parameters.
- Pruning redundant network connections.
- Quantization of weights while preserving accuracy.

The focus of this tutorial is the BinaryConnect paper, which falls in the third category. The paper proposes a training procedure to learn binarized weights (+1, -1) as opposed to full precision weights (32bits or 64bits) with minimal loss in precision.

## 2. Objective
We will not be replicating the results of the paper. Rather, this will serve as a proof-of-concept for the ...umm, concept. These are the concession that we will be making in the interest of time.
- Working with MNIST and only MNIST.
- Training a VERY simple model (logistic regression).
- Using slow and un-optimised Python.

The last point it actually more important that you might think. The true potential of such approaches is unlocked while using specialised hardware and software which actually exploits the single bit weights. But you won't get to see that with Python because I will still use 32 bit integers to represent +1 and -1. So there!

## 3. Some Theory
For those unfamiliar with back-propagation, there's a excellent post on the topic by [Michael Nielsen](http://neuralnetworksanddeeplearning.com/chap2.html). Here's a <INSERT_FAVORITE_BIG_NUMBER> foot overview using logistic regression model.

### 3.1. Model for multi-class logistic regression
I define a multi-class classification problems as follows:
$$
\begin{align}
X & \sim input\ matrix[batch, features] \\
W & \sim weight\ matrix[features, classes] \\
\textbf{b} & \sim bias\ vector[1, classes]
\end{align}
$$

The logistic model $f$ is defined as
$$
f(X, W, \textbf{b}) = softmax(X.W + \textbf{b})
$$
where
$$
softmax(v_{ij}) = \frac{e^{v_{ij}}}{\sum_k e^{v_{ik}}}
$$

$f$ returns the *logits*, which are the model output, which in this case will be a probability distribution for every sample against every possible class. We can calculate the error from the correct class label using any number of loss/distance measures. In the spirit of needlessly complicating things, let's continue with [cross-entropy](https://en.wikipedia.org/wiki/Cross_entropy). Amongst other things, a cross-entropy as an error measure is helpful as it allows weight updates even when the activations are close to saturation, i.e. when the error gradient is very close to 0. Additionally, the combination of a softmax output with cross entropy error gives a very compact and elegant error gradient w.r.t. the input to the softmax, which in this case is $(X.T + \textbf{b})$.

We define cross entropy loss $XE$ for a target distribution $\textbf{t}$ and predicted distribution $\textbf{p}$ as  follows
$$
XE(\textbf{t}, \textbf{p}) = - \sum_i t_i log(p_i)
$$
where $i$ denotes the class(or dimension) index.

### 3.2. Calculating error gradients

Training the network requires calculating the error gradients w.r.t the parameters: $W, \textbf{b}$. And since the output is a function of the parameters, we can just keep applying the chain-rule repeatedly to get the error gradients w.r.t. the parameters, starting from the gradient w.r.t. the output.

This repeated and recursive transfer of the error is called *backpropagating the error*. If you're not happy with this definition, raise a PullRequest with a better one. 

At this point, I will go over the notation and indices once again because it gets pretty gnarly. 

- $X$: uppercase letters represent row-ordered matrices.
- $\textbf{x}$: bold lowercase letters represent vectors. Here $\textbf{x}$ is a single sample of the batched data in $X$.
- $x_{ij}$: lowercase normal font letters represent single values. $x_{ij}$ represents the $j^{th}$ column element for the $i^{th}$ row in the input matrix $X$.

**Gradients for logits $l$**

$$
\begin{align}
\frac{\partial XE}{\partial l_{ij}} &= \frac{\partial XE(\textbf{t}, \textbf{l}_i)}{\partial l_{ij}}
\end{align}
$$

Note that the gradient is being calculated for a single element $(i, j)$ of the logits matrix $L$. For clarity, I'm assuming there's only one sample in the batch (i.e. $i=1$) which leaves us with logits vector $\textbf{l}_i$. I have omitted the first(sample or batch) index $i$ henceforth.

Now, with that out of the way, let's continue.
$$
\begin{align}
\frac{\partial XE}{\partial l_j} &= \frac{\partial XE(\textbf{t}, \textbf{l})}{\partial l_j} \\ \\
&= \frac{ \partial(- \sum_k t_k log (l_k)) }{\partial l_j} \\ \\
&= \frac{ - \sum_k t_k \partial log (l_k) }{\partial l_j}
\end{align}
$$

As a general rule, while introducing any new function or expression that involves indices, make it a point to declare new index placeholders - it will lead to fewer errors and a lot less confusion.



Here, $k$ iterates over all class labels. Expanding the sigma,  for $m$ class labels, we will have $m$ different partial derivative terms. And for exactly one term, the numerator and denominator will have the same index. Concretely, for one term, $k = j$ and for all other $n-1$ terms, $k \neq j$. 

\begin{align}
\Rightarrow \frac{\partial XE}{\partial l_j} &= - t_{k=j}\frac{\partial log(l_{k=j})}{\partial l_j} -  \frac{  \sum_{k \neq j} t_k \partial log (l_k) }{\partial l_j}
\end{align}

It might be tempting to remove the second term in haste, by observing that the indices do not match and concluding that the derivative w.r.t. $l_j$ will be zero. But, recall the definition of $softmax$, where we divide by the sum of all label probabilities. 

**Gradients for linear transformation $Z$**

I'm introducing a new variable $Z$ defined as 
$$ Z = X.W + \textbf{b} $$
which links to the logits as follows
$$ l_{j} = softmax(z_{j}) $$

Therefore, to calculate the derivatives for the parameters, we need to calculate the gradient w.r.t. $z_j$ using backpropagation.

$$
\begin{align}
\frac{\partial XE}{\partial z_j} &= \frac{\partial XE}{\partial l_s} \times  \frac{\partial l_s}{\partial z_j} \\ \\
where\ \frac{\partial l_s}{\partial z_j} &= \frac{\partial softmax(z_s) }{\partial z_j} \\ \\
and \ softmax(z_s) &= \frac{e^{z_s}}{\sum_v e^{z_v}}
\end{align}
$$

Recall the definition of logit vector $l_s$ in terms of the softmax. 

$$
l_s = softmax(z_s) =  \frac{e^{z_s}}{\sum_v e^{z_v}}
$$

While taking the derivative of the denominator in the $softmax$ w.r.t. $z_j$, remeber that $v$ iterates over all label indices and one of them will be equal to $j$. Only for that index, will the deivative exist and equal to $e^{z_j}$; for all other indices, the derivative will be 0. With that, we can compute the gradient w.r.t. $z_j$

$$
\begin{align}
\frac{\partial l_s}{\partial z_j} &= \frac{\partial q_s }{\partial z_j} \\ \\
= \frac{ e^{z_s}  \sum_v e^{z_v}  - e^{z_s} e^{z_j}}{  (\sum_v e^{z_v})^2 } \tag {for s$ = j$} =&  \frac{ e^{z_s}(  \sum_v e^{z_v}  - e^{z_j})}{  (\sum_v e^{z_v})^2 } \\ \\
and\ \  \ & \frac{ 0  - e^{z_s} e^{z_j}}{  (\sum_v e^{z_v})^2 } \tag{for s$ \neq j$} \\ \\
\\
For \ (s=j):&\  \frac{ e^{z_s} }{ \sum_v e^{z_v} } \bigg(  1 -  \frac{ e^{z_j} }{ \sum_v e^{z_v} } \bigg)  = l_s(1 - l_j) \\ \\
For \ (s \neq j):&\  - \frac{ e^{z_s} }{ \sum_v e^{z_v} } \frac{ e^{z_j} }{ \sum_v e^{z_v} }   = - l_s  l_j \\ \\
\end{align} \\ \\
$$

I really, *really* tried trying to use those large curly braces you get with `begin{cases}` in Latex, but they just weren't working well with `\frac` in MathJax. Anyways, now we can compute the error gradients w.r.t. $z_j$.

$$
\begin{align}
\frac{\partial XE}{\partial z_j} &= \frac{\partial XE}{\partial l_s} \times  \frac{\partial l_s}{\partial z_j} \\ \\
&= - \bigg(   t_{k=s}\frac{\partial log(l_{k=s})}{\partial l_s}  +  \frac{  \sum_{k \neq j} t_k \partial log (l_k) }{\partial l_s}   \bigg) \times \frac{\partial l_s}{\partial z_j} \\ \\
&= - \bigg(   \frac{t_{k=s}}{l_{k=s}} \frac{\partial l_s}{\partial z_j}  +  \sum_{k \neq j} \frac{   t_k  }{ l_k }  \frac{\partial l_s}{\partial z_j}   \bigg)  \\ \\
\end{align}
$$
In the first term, index $k = s = j$, and in the second, $k = s \neq j$. Using the gradients computed before, we get

\begin{align}
For\ (k = s = j)&: \ \frac{t_{k=s = j}}{l_{k=s = j}} l_{s=j}(1 - l_j) = t_{k= j} (1 - l_j) \\ \\
For\ (k = s \neq j)&: \ \sum_{k \neq j} \frac{   t_k  }{ l_k } \times (-l_{s=k \neq j} l_j)  = - \sum_{k \neq j} t_k l_j \\ \\
Adding\ both\ \ldots \\ \\
\frac{\partial XE}{\partial z_j} &= - \bigg(   t_{k = j} (1 - l_j) - \sum_{k \neq j} t_k l_j   \bigg) \\ \\
&= -\bigg(   t_{k=j} - l_j t_{k=j}    - l_j \sum_{k \neq j} t_k   \bigg) & \text{since logit $l_j$ does not depend on index $k$ } \\ \\
&= -\bigg(   t_{k=j} - l_j \bigg( \sum_{k \neq j} t_k  + t_{k=j} \bigg) \bigg) \\ \\
&= - (t_{k=j} - l_j) & \text{$t_k$ is a probability distribution: $\sum_k t_k = 1$} \\ \\
&= l_j - t_j
\end{align}

Thus, the final error gradient w.r.t. the linear transformation of the inputs and the parameters is the difference between the predicted value and the target value. With that, we get to the final gradient calculations w.r.t. the parameters of the model.

**Gradients for the model parameters W, b**

$Z$ is a linear transformation of the input $X$, weight $W$ and bias $\textbf{b}$. Following the dot product, an element $z_j$ can be written as 
$$
z_j = \sum_i  x_i \times  w_{ij}
$$
Thus, it's derivative w.r.t. $w_{ij}$ will be $x_i$. Therefore
$$
\begin{align}
\frac{\partial XE}{\partial w_{ij}} &= \frac{\partial XE}{\partial z_j} \frac{\partial z_j}{\partial w_{ij}} \\ \\
&= (l_j - t_j) \times x_i \\ \\
\end{align}
$$
The derivative w.r.t. an element $b_j$ of the bias vector will be 1. Therefore
$$
\begin{align} \\
\frac{\partial XE}{\partial b_{j}} &= \frac{\partial XE}{\partial z_j} \frac{\partial z_j}{\partial b_{j}} \\ \\
&= (l_j - t_j) \times 1 \\
\end{align}
$$

**Vector calculations**

The calculations suggest that all operations are performed at an element level. Instead of looping, we can vectorise these operations and calculate the error gradients for the entire matrix - or in this case where the batch size is 1, the entire vector.

$$
\begin{align}
\frac{\partial XE}{\partial Z} &= L - T \\ \\
\frac{\partial XE}{\partial W} &=  X^T. (L - T) \\ \\
\frac{\partial XE}{\partial \textbf{b}} &=  \sum_i(l_{ij} - t_{ij}) \\ 
where \\
L &: \text{logits matrix of size (BatchSize x NumClasses)} \\
T &: \text{target matrix of size (BatchSize x NumClasses)} \\
Z &: \text{ $X.W + \textbf{b}$ (BatchSize x NumClasses)} \\
X &: \text{input matrix of size (BatchSize x NumFeatures)} \\
W &: \text{weight matrix of size (NumFeatures, NumClasses)} \\
\textbf{b} &:  \text{bias vector of size (1, NumClasses)}
\end{align}
$$
A way to interpret the last equation is that we are summing the error gradient over all possible samples, for every class.

## 4. Forward Pass

```python
## Imports.
import numpy as np
np.random.seed(1234) ## For reproducibility.
```

For the forward pass, we'll use 2 functions. The first to compute the logits `def logistic_model(inputs, weights, bias)`  and the second to compute the cross entropy loss `def ce_loss(output, target)` .

```python
## For the logits.
def logistic_model(inputs, weights, bias):
    z = np.dot(x, w) + b
    z = z - np.max(z, axis=1, keepdims=True) ## Safe softmax!
    _e = np.exp(z)
    logits = np.divide(_e, np.sum(_e, axis=1, keepdims=True))
    return logits

## For the cross entropy.
def ce_loss(output, target):
    return -1.0 * np.mean( 
        np.sum(np.multiply(target, np.log(output)), axis=1) 
    )
```

There are some deviations from the derivation for practical purposes:
- Since softmax involves an exponentiation, the numbers (and the denominator) can explode for even moderately large $z_j$ values. To scale the values down, you subtract the largest value for every row: `z = z - np.max(z, axis=1, keepdims=True) ## Safe softmax!`. This makes no difference to the softmax computation because we take the exponent and then divide by the sum of the exponent.

$$
\begin{align}
a &= [1, 2, 3] \\ \\
b &= e^{a},\ \ b_1 = e^1 \\ \\ 
c_1 &= \frac{e^1}{(e^1 + e^2 + e^3)} \\ \\
&Multiplying\ and\ dividing\ by\ e^{-3} \\ \\ 
c_1 &= \frac{e^{-2}}{(e^{-2} + e^{-1} + e^0)} \\ \\
&Thus \ b,a\ could\ be\ written\ as \\ \\
b &= [e^{-2} + e^{-1} + e^0] \\ \\
a &= [-2, -1, 0]
\end{align}
$$

- The final loss value after every forward pass will be an average loss over all samples. Hence the `np.mean(...)`  in the cross entropy loss function. Since we are dividing by the batchsize here, we will need to scale the error gradient accordingly. 

$$
\begin{align}
&\frac{\partial XE}{\partial Z} = \frac{1}{n}(L - T) \\ \\
&where\ \text{$n$: BatchSize}
\end{align}
$$